In [228]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m, WGS84
gcsgp = GCSGeoPandas()
from shared_utils import gtfs_utils_v2
from shared_utils.rt_utils import show_full_df

# Explore Thruway Bus (and state-suppoted rail) data availability

In [2]:
analysis_date = '2025-08-18'

In [110]:
ca = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/ca_boundary.parquet')

## New SJJPA San Joaquins feed

In [3]:
query = '''
SELECT * from cal-itp-data-infra.mart_transit_database.dim_gtfs_service_data
WHERE _is_current
LIMIT 1000
'''

In [4]:
df = query_sql(query)

In [5]:
sanj = df.query('name.str.contains("Amtrak San J")')

In [6]:
sanj

,key,name,source_record_id,service_key,gtfs_dataset_key,customer_facing,category,fares_v2_status,manual_check__fixed_route_completeness,manual_check__demand_response_completeness,_is_current,_valid_from,_valid_to
713,c9364f7e1d226a2a1b57de51ec4375ad,Amtrak San Joaquins – Amtrak San Joaquins Sche...,recBvVy7cJzjXBjeD,b3a26733da02debeb0e23e30c8d8e928,524ea6209600e9a2de34a02cf9068729,True,primary,[Unknown],Unknown,Unknown,True,2025-08-14 00:00:00+00:00,2098-12-31 23:59:59.999999+00:00


In [7]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

In [8]:
sanj = feeds.query('name.str.contains("Amtrak San Joaquins")')
sanj

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,regional_feed_type,type
82,a04bf510313ab925aa37eea637a18bec,2025-08-18,d9e1e77d0754b712fc608741ae3836f5,America/Los_Angeles,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,524ea6209600e9a2de34a02cf9068729,Amtrak San Joaquins Schedule,None,schedule


In [9]:
gtfs_utils_v2.get_shapes?

Signature:
gtfs_utils_v2.get_shapes(
    selected_date: Union[str, datetime.date],
    operator_feeds: list[str] = [],
    shape_cols: list[str] = [],
    get_df: bool = True,
    crs: str = 'EPSG:4326',
    custom_filtering: dict = None,
) -> geopandas.geodataframe.GeoDataFrame
Docstring:
Query fct_daily_scheduled_shapes.

Must supply a list of feed_keys returned from
schedule_daily_feed_to_gtfs_dataset_name() or subset of those results.
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py
Type:      function

In [10]:
shape_cols = ['feed_key', 'shape_array_key']

In [11]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [12]:
sanj_shapes

,feed_key,shape_array_key,geometry
0,d9e1e77d0754b712fc608741ae3836f5,e95c0478ad09606a277e2f9a0cdccfbd,"LINESTRING (-118.23725 34.05667, -118.23713 34..."
1,d9e1e77d0754b712fc608741ae3836f5,3080cb80496b689cbc3f2fce8887a714,"LINESTRING (-122.02465 36.97091, -122.02483 36..."
2,d9e1e77d0754b712fc608741ae3836f5,e1db4d90fdd39a9b668acab298cadea7,"LINESTRING (-121.28492 37.94505, -121.28498 37..."
3,d9e1e77d0754b712fc608741ae3836f5,6b08e36b8b57f04d365cb3495703df1d,"LINESTRING (-119.00770 35.37218, -119.00765 35..."
4,d9e1e77d0754b712fc608741ae3836f5,143691f5856233e9c0a99160e30a9b15,"LINESTRING (-121.50113 38.58228, -121.50109 38..."
...,...,...,...
63,d9e1e77d0754b712fc608741ae3836f5,4d1d764f99b6f3abaecd0ae238776088,"LINESTRING (-122.27147 37.79379, -122.27169 37..."
64,d9e1e77d0754b712fc608741ae3836f5,e2c04f958d1a6a3ab444b5590924ad27,"LINESTRING (-119.00770 35.37218, -119.00762 35..."
65,d9e1e77d0754b712fc608741ae3836f5,ef9cbdb19e2f885fb77ba9da12ed3d3e,"LINESTRING (-122.29181 37.84045, -122.29161 37..."
66,d9e1e77d0754b712fc608741ae3836f5,82db3c1be8f398c6f49d9db438eaa443,"LINESTRING (-122.39850 37.78850, -122.39832 37..."


In [13]:
gtfs_utils_v2.get_trips?

Signature:
gtfs_utils_v2.get_trips(
    selected_date: Union[str, datetime.date],
    operator_feeds: list[str] = [],
    trip_cols: list[str] = [],
    get_df: bool = True,
    custom_filtering: dict = None,
) -> Union[pandas.core.frame.DataFrame, siuba.sql.backend.LazyTbl]
Docstring:
Query fct_scheduled_trips

Must supply a list of feed_keys returned from
schedule_daily_feed_to_gtfs_dataset_name() or subset of those results.
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py
Type:      function

In [14]:
trip_cols = ['trip_instance_key', 'trip_id', 'trip_short_name',
'base64_url', 'feed_key', 'name',
'regional_feed_type', 'gtfs_dataset_key', 'service_date',
'direction_id', 'block_id', 'route_key',
'route_id', 'route_type', 'route_short_name',
'route_long_name', 'route_desc', 'agency_id',
'network_id', 'shape_array_key', 'shape_id'
]

In [15]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=trip_cols)

In [16]:
# sanj_trips.query('route_type == "2"')

In [17]:
sanj_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

,,trip_instance_key
route_short_name,route_long_name,
ACETrain,ACE Train,8
AMSJ2,Amtrak San Joaquins BFD OKJ,12
Route 1,Fresno/Bekersfield - Los Angeles- SantaAna - Oceanside- San Diego,20
Route 10,Santa Barbara - Las Vegas,2
Route 15,Mammoth Lakes- Yosemite National Park,4
Route 15a,Merced - Yosemite National Park,12
Route 15b,Fresno - Yosemite National Park,6
Route 18,Santa Maria - San Luis Obispo - Handford,2
Route 19,Berkersfield - Inland Empire,4


In [18]:
sanj_shapes_trip_info = sanj_shapes.merge(sanj_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [234]:
# sanj_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

In [20]:
sanj_trips.query('route_short_name == "Route 99"')

,trip_instance_key,trip_id,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,service_date,direction_id,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id
57,9d9dae8395b78521dc2067ffc4fee6cd,6611,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
58,f34ce074ba43c8d8326cbbf80ef9f57d,6619,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
59,97c347c91d8733658a03d0ba680cb470,6617,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
60,bfa4ea4f854d9bac2537be504ee0e9e4,6615,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
61,eeed34ed58303fa1c5d4f1065b9a571e,6613,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,1,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,ef9cbdb19e2f885fb77ba9da12ed3d3e,f1x5
83,c0b3951c7daedca3c122bb2423d5a41b,6616,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
84,b2a88bb09ebd891178e8d29e4fe97a31,6610,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
85,978012028f8316de61c45e5ec928f705,6668_w,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
86,75bb0a711792d09aac78643b7dd46cd6,6614,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu
87,963b3d03c6e386332149c6e8afe740b2,6612_w,None,aHR0cHM6Ly9kMzR0aXc2NG41ejRvaC5jbG91ZGZyb250Lm...,d9e1e77d0754b712fc608741ae3836f5,Amtrak San Joaquins Schedule,None,524ea6209600e9a2de34a02cf9068729,2025-08-18,0,...,15dd7837f8f1ed260bc41eeecf20e935,99,3,Route 99,Oakland-Emeryville-San Francisco,None,SJRRC,None,82db3c1be8f398c6f49d9db438eaa443,p5wu


## Capitol Corridor

In [21]:
cc_feed = feeds.query('name.str.contains("Capitol")')

In [22]:
cc_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=cc_feed.feed_key, shape_cols=shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [23]:
cc_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=cc_feed.feed_key, trip_cols=trip_cols)

In [24]:
cc_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

,,trip_instance_key
route_short_name,route_long_name,
CC,Capitol Corridor,28
SF,Shuttle_SF,26
Shuttle,Shuttle_Auburn,10


In [25]:
cc_shapes_trip_info = cc_shapes.merge(cc_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [233]:
# cc_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

## Pacific Surfliner (via Amtrak National feed)

In [196]:
surf_feed = feeds.query('name.str.contains("Amtrak Schedule")')

In [197]:
surf_feed

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,regional_feed_type,type
108,ce215428af7326a7911fc716e8fe47c7,2025-08-18,439b3e1dddd1e8ce637692e626cb9799,America/New_York,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,6f6e2c08a2fc1b57ad3cb6fa7699694b,Amtrak Schedule,None,schedule


In [198]:
surf_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=surf_feed.feed_key, shape_cols=shape_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [199]:
stop_cols = ['feed_key', 'stop_id', 'stop_name']

In [200]:
surf_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=surf_feed.feed_key, stop_cols=stop_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [201]:
surf_stops = surf_stops.to_crs(CA_NAD83Albers_m)
ca = ca.to_crs(CA_NAD83Albers_m)

surf_stops = surf_stops.clip(ca)

In [202]:
surf_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=surf_feed.feed_key, trip_cols=trip_cols)

In [203]:
stop_time_cols = ['feed_key', 'trip_id', 'stop_id',
                 'stop_sequence', 'arrival_time', 'departure_time',
                 'arrival_sec', 'departure_sec']

In [204]:
surf_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=surf_feed.feed_key, get_df=True,
                                      trip_df = surf_trips, stop_time_cols=stop_time_cols)

/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.11/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [205]:
surf_trips = surf_trips.query('route_long_name == "Amtrak Thruway Connecting Service"')

In [206]:
surf_merged = surf_trips.merge(surf_st, on = ['feed_key', 'trip_id']).merge(surf_stops, on = ['feed_key', 'stop_id'])

### CA Thruway in Amtrak National Feed:

#### `route_id` 41080

* ['Chico Amtrak', 'Red Bluff Amtrak Bus Stop',
       'Redding Amtrak Bus Stop']

#### 42896

* ['Bakersfield', 'Uc Santa Barbara Amtrak Bus Stop', 'Santa Barbara',
       'Las Vegas', 'Las Vegas Amtrak Bus Stop',
       'Barstow Amtrak Bus Stop', 'Barstow', 'Mojave Amtrak Bus Stop',
       'Tehachapi Amtrak Bus Stop', 'Fillmore Amtrak Bus Stop',
       'Santa Paula Amtrak Bus Stop', 'Oxnard', 'Ventura Amtrak',
       'Carpinteria Amtrak']

#### 42954

* strange one, nationwide stops including CA points

#### `route_id` not that useful, look for trip patterns...



In [207]:
trip_id_stops = surf_merged[['trip_id', 'stop_name']].groupby('trip_id')['stop_name'].unique().map(lambda x: frozenset(x))

In [208]:
trip_id_stops

trip_id
183836    (Truckee Amtrak Station, Roseville Amtrak, Roc...
183852    (Truckee Amtrak Station, Roseville Amtrak, Roc...
183877                       (Roseville Amtrak, Sacramento)
183910    (Elk Grove Amtrak Bus Stop, Stockton, Sacramento)
183911    (Chico Amtrak, Stockton, Sacramento Amtrak Bus...
                                ...                        
195441                          (San Francisco, Emeryville)
195446                          (San Francisco, Emeryville)
198193    (San Diego, Solana Beach Amtrak Station, Baker...
198207    (San Diego, Solana Beach Amtrak Station, Baker...
198426    (Garberville Amtrak Bus Stop, Cloverdale Amtra...
Name: stop_name, Length: 128, dtype: object

In [209]:
trip_patterns = trip_id_stops.value_counts() # count unique trip patterns...

In [210]:
# trip_patterns

### Surfliner-associated Trip Patterns

https://www.pacificsurfliner.com/plan-your-trip/connections/bus-connections/

In [211]:
sb_oakland = trip_patterns.index[2]
sb_oakland

frozenset({'Buellton Amtrak Bus Stop',
           'Emeryville',
           'Grover Beach Amtrak',
           'King City Amtrak Bus Stop',
           'Oakland',
           'Paso Robles Amtrak Station',
           'Salinas',
           'San Francisco',
           'San Jose',
           'San Luis Obispo',
           'San Luis Obispo Amtrak Bus Stop',
           'Santa Barbara',
           'Santa Maria Amtrak Bus Stop',
           'Solvang Amtrak Bus Stop',
           'Uc Santa Barbara Amtrak Bus Stop'})

In [212]:
fullerton_indio = trip_patterns.index[21]

In [213]:
fullerton_indio

frozenset({'Cabazon Amtrak Bus Stop',
           'Fullerton Amtrak Station',
           'Indio Amtrak Bus Stop',
           'La Quinta Amtrak Bus Stop',
           'Palm Desert Amtrak Bus Stop',
           'Palm Springs Airport Amtrak Bus Stop',
           'Palm Springs Amtrak Bus Stop',
           'Riverside'})

In [214]:
fullerton_palm_springs = trip_patterns.index[24]

In [215]:
fullerton_palm_springs

frozenset({'Cabazon Amtrak Bus Stop',
           'Fullerton Amtrak Station',
           'Palm Springs Airport Amtrak Bus Stop',
           'Palm Springs Amtrak Bus Stop',
           'Riverside'})

In [216]:
surfliner_routes_from_trip_patterns = {fullerton_palm_springs: 'Fullerton - Palm Springs', fullerton_indio: 'Fullerton - Indio', sb_oakland: 'Santa Barbara - Oakland'}

In [217]:
trips_with_pattern = trip_id_stops.reset_index().rename(columns={'stop_name':'trip_pattern'})
trips_with_pattern = trips_with_pattern.assign(manual_route_name = trips_with_pattern.trip_pattern.map(
    lambda x: surfliner_routes_from_trip_patterns[x] if x in surfliner_routes_from_trip_patterns.keys() else None)
                                              )

In [218]:
trips_with_pattern = trips_with_pattern.dropna() #  trip_id for Surfliner-associated routes...
trips_with_pattern

,trip_id,trip_pattern,manual_route_name
25,183945,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland
26,183952,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Indio
27,183953,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Palm Springs
28,183954,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Indio
29,183955,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Palm Springs
58,186749,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland
59,186752,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland
63,187892,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland
64,187895,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland
118,194597,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland


In [219]:
trips_with_pattern = trips_with_pattern.merge(surf_trips, on='trip_id')

In [220]:
trips_with_pattern

,trip_id,trip_pattern,manual_route_name,trip_instance_key,trip_short_name,base64_url,feed_key,name,regional_feed_type,gtfs_dataset_key,...,route_key,route_id,route_type,route_short_name,route_long_name,route_desc,agency_id,network_id,shape_array_key,shape_id
0,183945,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland,a10ce2d0ea671f4bb2126a5fea5971b0,4770,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
1,183952,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Indio,79efd275403e0d37af96486aa03f63e7,4967,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
2,183953,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Palm Springs,ddd40f90568c8e80573a57be466f7b55,4968,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
3,183954,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Indio,d3e05855b0de100ce4333abc29009811,4984,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
4,183955,"(Riverside, Cabazon Amtrak Bus Stop, Palm Spri...",Fullerton - Palm Springs,8f01fe5bef6636dc6bc6de1e617d8937,4985,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
5,186749,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland,f2ba0dae77b0c28fc9e499a1f7348f75,4765,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
6,186752,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland,15749713f4a8c917167dec608a0e97e7,4769,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
7,187892,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland,e950613d5bcc9317e6f867a57b0b3bad,4784,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
8,187895,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland,764030a53c8e5456c4e3111eab7f9390,4785,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None
9,194597,"(King City Amtrak Bus Stop, Santa Maria Amtrak...",Santa Barbara - Oakland,bb08a8f14b5611e82e87c0377e4ebedd,4790,aHR0cHM6Ly9jb250ZW50LmFtdHJhay5jb20vY29udGVudC...,439b3e1dddd1e8ce637692e626cb9799,Amtrak Schedule,None,6f6e2c08a2fc1b57ad3cb6fa7699694b,...,2af23a5a0cb42e917ec20f684af8f0dd,42954,3,None,Amtrak Thruway Connecting Service,None,1232,None,None,None


In [223]:
#  unhelpful that these all have the same route_id!
trips_with_pattern[['route_id', 'route_long_name', 'trip_instance_key']].groupby(['route_long_name', 'route_id']).count()

,,trip_instance_key
route_long_name,route_id,
Amtrak Thruway Connecting Service,42954,10


In [226]:
surf_st_merged = surf_stops.merge(surf_st, on = ['feed_key', 'stop_id']).merge(trips_with_pattern, on = ['feed_key', 'trip_id'])

In [231]:
view_cols = ['manual_route_name', 'stop_id', 'stop_name',
             'trip_id', 'stop_sequence', 'arrival_time',
             'departure_time', 'route_id', 'geometry']

## A derived timetable of Pacific Surfliner-associated thruways in the Amtrak Nationwide feed

Daily Trips:

* 1 Indio - Fullerton
* 1 Fullerton - Indio
* 1 Fullerton - Palm Springs Airport
* 1 Palm Springs Airport - Fullerton
* 4 Oakland - Santa Barbara
* 4 Santa Barbara - Oakland

In [232]:
show_full_df(surf_st_merged.sort_values(['manual_route_name', 'trip_id', 'stop_sequence'])[view_cols])

,manual_route_name,stop_id,stop_name,trip_id,stop_sequence,arrival_time,departure_time,route_id,geometry
1,Fullerton - Indio,IDO,Indio Amtrak Bus Stop,183952,1,9:50:00,9:50:00,42954,POINT (349165.835 -470852.616)
0,Fullerton - Indio,LQT,La Quinta Amtrak Bus Stop,183952,2,10:00:00,10:00:00,42954,POINT (343917.412 -471490.693)
2,Fullerton - Indio,PDC,Palm Desert Amtrak Bus Stop,183952,3,10:15:00,10:15:00,42954,POINT (333797.307 -470245.871)
4,Fullerton - Indio,PSP,Palm Springs Airport Amtrak Bus Stop,183952,4,10:45:00,10:45:00,42954,POINT (323166.631 -459735.855)
3,Fullerton - Indio,PSS,Palm Springs Amtrak Bus Stop,183952,5,10:50:00,10:50:00,42954,POINT (319786.685 -459789.359)
6,Fullerton - Indio,CBZ,Cabazon Amtrak Bus Stop,183952,6,11:20:00,11:20:00,42954,POINT (295987.993 -449877.275)
7,Fullerton - Indio,RIV,Riverside,183952,7,12:00:00,12:00:00,42954,POINT (243016.788 -445494.790)
5,Fullerton - Indio,FUL,Fullerton Amtrak Station,183952,8,13:05:00,13:05:00,42954,POINT (192200.181 -458594.647)
13,Fullerton - Indio,FUL,Fullerton Amtrak Station,183954,1,19:50:00,19:50:00,42954,POINT (192200.181 -458594.647)
15,Fullerton - Indio,RIV,Riverside,183954,2,20:45:00,20:45:00,42954,POINT (243016.788 -445494.790)
